This notebook requires pyfabm. For instructions on how to build and install pyfabm, see https://github.com/fabm-model/fabm/wiki/python

In [ ]:
import pyfabm
import numpy
import scipy.integrate

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
import matplotlib.pyplot

In [ ]:
model = pyfabm.Model('../fabm-gotm-npzd.yaml')

In [ ]:
# Configure environmental conditions and initialize the model
model.cell_thickness = 10  # cell thickness in m, used by getRates to scale surface and bottom fluxes
model.findDependency('surface_downwelling_photosynthetic_radiative_flux').value = 70  # W/m2
model.findDependency('downwelling_photosynthetic_radiative_flux').value = 10          # W/m2
assert model.start(), 'Model failed to start: %s' % pyfabm.getError()

In [ ]:
# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
def dy(y, t0):
    model.state[:] = y
    return model.getRates(t0)
t = numpy.arange(0, 200., 1)
y = scipy.integrate.odeint(dy, model.state, t*86400)

In [ ]:
# Plot results
matplotlib.pyplot.ioff()
fig = matplotlib.pyplot.figure()
ax = fig.gca()
ax.plot(t, y)
ax.legend([v.long_name for v in model.state_variables])
ax.grid()
ax.set_xlabel('time (d)')
ax.set_ylabel(model.state_variables[0].units)
fig.show()
